In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input


def build_model(num_classes):
    '''
    Создание и компиляция модели сверточной нейронной сети для классификации изображений.

    Параметры:
        num_classes (int): Количество классов в задаче классификации.

    Возвращает:
        Sequential: Модель Keras, готовая к обучению.

    Описание модели:
        1. Input layer: Принимает изображения размером 28x28 пикселей с одним каналом (черно-белые).
        2. Conv2D layer: Сверточный слой с 32 фильтрами размером 3x3 и функцией активации ReLU.
        3. MaxPooling2D layer: Слой максимального пулинга с размером пула 2x2.
        4. Dropout layer: Слой исключения с коэффициентом 0.5 для предотвращения переобучения.
        5. Flatten layer: Преобразует многомерные карты признаков в одномерный вектор.
        6. Dense layer: Полносвязный слой с 128 нейронами и функцией активации ReLU.
        7. Output Dense layer: Выходной полносвязный слой с количеством нейронов, равным количеству классов, и функцией активации Softmax для многоклассовой классификации.

    Компиляция модели:
        - Оптимизатор: Adam.
        - Функция потерь: categorical_crossentropy.
        - Метрика: accuracy.
    '''
    model = Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.5),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


# Загрузка данных и меток из файла
data = np.load('fonts_dataset.npz')
X_train, X_test, y_train, y_test = data['X_train'], data['X_test'], data['y_train'], data['y_test']
labels = data['labels']

# Определение количества классов (шрифтов)
num_classes = len(set(labels))
model = build_model(num_classes)

# Печать сводки модели
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       692,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 694,607 (2.65 MB)

 Trainable params: 694,607 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.metrics import classification_report, confusion_matrix


def train_model(model, X_train, y_train, X_test, y_test, epochs=15, batch_size=128):
    '''Обучение модели и вывод основных метрик классификации'''
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_test, y_test), verbose=2)
    
    # Логгирование результатов в DataFrame
    history_df = pd.DataFrame(history.history)
    history_df.to_csv('training_log.csv', index=False)

    # Оценка модели
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test loss: {loss:.3f}, Test accuracy: {accuracy:.3f}")
    
    # Предсказание классов для тестовых данных
    y_pred = np.argmax(model.predict(X_test), axis=1)

    # Вывод отчета о классификации и матрицы ошибок
    print("Classification Report:")
    print(classification_report(y_test_encoded, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test_encoded, y_pred))
    
    model.save("font_recognition_model.keras")

    return model


# Подготовка данных
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train.ravel())
y_test_encoded = label_encoder.transform(y_test.ravel())
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)
    
# Нормализация данных
X_train_normalized = X_train.astype('float32') / 255.0
X_test_normalized = X_test.astype('float32') / 255.0
X_train_normalized = np.expand_dims(X_train_normalized, axis=-1)
X_test_normalized = np.expand_dims(X_test_normalized, axis=-1)

# Обучение модели
model = train_model(model, X_train_normalized, y_train_onehot, X_test_normalized, y_test_onehot)

Epoch 1/15
79/79 - 2s - 20ms/step - accuracy: 0.6157 - loss: 1.2603 - val_accuracy: 0.8427 - val_loss: 0.5592
Epoch 2/15
79/79 - 1s - 14ms/step - accuracy: 0.8905 - loss: 0.4074 - val_accuracy: 0.9448 - val_loss: 0.2429
Epoch 3/15
79/79 - 1s - 13ms/step - accuracy: 0.9401 - loss: 0.2295 - val_accuracy: 0.9584 - val_loss: 0.1580
Epoch 4/15
79/79 - 1s - 14ms/step - accuracy: 0.9613 - loss: 0.1571 - val_accuracy: 0.9744 - val_loss: 0.1065
Epoch 5/15
79/79 - 1s - 13ms/step - accuracy: 0.9675 - loss: 0.1193 - val_accuracy: 0.9884 - val_loss: 0.0770
Epoch 6/15
79/79 - 1s - 14ms/step - accuracy: 0.9731 - loss: 0.0970 - val_accuracy: 0.9888 - val_loss: 0.0602
Epoch 7/15
79/79 - 1s - 13ms/step - accuracy: 0.9799 - loss: 0.0759 - val_accuracy: 0.9884 - val_loss: 0.0523
Epoch 8/15
79/79 - 1s - 13ms/step - accuracy: 0.9817 - loss: 0.0617 - val_accuracy: 0.9904 - val_loss: 0.0408
Epoch 9/15
79/79 - 1s - 13ms/step - accuracy: 0.9843 - loss: 0.0552 - val_accuracy: 0.9948 - val_loss: 0.0354
Epoch 10/1